### Amazon Datascraper and Handler

In [283]:

import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import requests

In [284]:
def search_am(phrase):
    link="https://www.amazon.in/s?k="
    l_end="&ref=nb_sb_noss"
    phrase_w= phrase.replace(' ','+')
    link_full=link+phrase_w+l_end
    #print(link_full)
    
    driver = webdriver.Chrome()
    
    wait = WebDriverWait(driver, 5)
    driver.get(link_full)
    
    names_f=[]
    names=driver.find_elements_by_tag_name("a")
    i=0
    for name in names:
        className = name.get_attribute('class')
        if className=='a-link-normal a-text-normal':
            names_f.append(name)
            i+=1
       
            
    
    links=[]
    for i in names_f:
        temp= i.get_attribute('href')
        links.append(temp)
        
    driver.quit()
    return links
    


In [285]:
def get_element_dets(link):
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 2)
    driver.get(link)
    title_o= driver.find_elements_by_id("productTitle")
    title=title_o[0].text
    
    number_o= driver.find_elements_by_id("acrCustomerReviewText")
    try:
        popularity=(number_o[0].text)
    except:
        popularity='0'
    rate=driver.find_elements_by_css_selector("#reviewsMedley > div > div.a-fixed-left-grid-col.a-col-left > div.a-section.a-spacing-none.a-spacing-top-mini.cr-widget-ACR > div.a-fixed-left-grid.AverageCustomerReviews.a-spacing-small > div > div.a-fixed-left-grid-col.aok-align-center.a-col-right > div > span > span")
    try:
        rate_o=(rate[0].text).split(' ')[0]
    except:
        rate_o='0'
    feat_f=[]
    tag=[]
    value=[]
    #features=driver.find_elements_by_css_selector("#feature-bullets > ul > li > span")
    #for f in features:
    #    feat_f.append(f.text)
    price=0
    try:
        tag_o=driver.find_elements_by_tag_name('th')
        for name in tag_o:
            className = name.get_attribute('class')
            if className=='a-color-secondary a-size-base prodDetSectionEntry':
                tag.append(name.text)
    
        value_o=driver.find_elements_by_tag_name('td')        
        for name in value_o:
            className = name.get_attribute('class')
            if className=='a-size-base':
                value.append(name.text)
        i=0
        while i<len(value):
            t=str(tag[i])+':'+str(value[i])
            feat_f.append(t)
            i+=1
    except:
        feat_f=[':'] 
    try:
        price_o= driver.find_elements_by_id("priceblock_ourprice")
        for name in price_o:
            className = name.get_attribute('class')
            if className=='a-size-medium a-color-price priceBlockBuyingPriceString':
                price=(name.text)
                break
    except:
        price=0
    #price=price_o.text
    
    feedbacks=driver.find_elements_by_tag_name("a")
    
    feedback_f=[]
    for feed in feedbacks:
        className = feed.get_attribute('class')
        if className=='a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold':
            feedback_f.append(feed.text)
            
    driver.quit()
    return feedback_f,title,rate_o,popularity,feat_f,price
    
    
    
    
    
            
        
   
    
    
    
    

In [286]:
def caller(phrase):
    links=search_am(phrase)
    data={}
    print(len(links))
    for link in links:
        data[link]={}
        feedback_f,title,rate,popularity,feat_f,price=get_element_dets(link)
        data[link]['feedback']=feedback_f
        data[link]['title']=title
        data[link]['rate']=rate
        data[link]['popularity']=popularity
        data[link]['features']=feat_f
        if isinstance(price, int):
            data[link]['price']=price
            
        else:
            
            data[link]['price']=price.split(' ')[1]
        #print(len(data))
    return data
        
    

In [287]:
##Done

### Popularity and Rating Based System

In [288]:
def assign_popularity_rating():
    with open('products.json', 'r') as openfile: 
  
   
        data = json.load(openfile) 
    temp=0
    for k in data.keys():
        p=int(data[k]['popularity'].split(' ')[0])
        r=float(data[k]['rate'])
        if p<50:
            temp=1
        elif p<100:
            temp=2
        elif p<150:
            temp=3
        else:
            temp=4
        score=(temp)
        data[k]['Popularity_Score']=score
        data[k]['Rating_Score']=r
        with open("products_mod.json", "w") as outfile: 
            json.dump(data, outfile) 

In [289]:
## Done

### Review Sentiment Based System

In [290]:
from textblob import TextBlob


In [291]:
def assign_sentiment_rating():
    with open('products_mod.json', 'r') as openfile: 
  
   
        data = json.load(openfile) 
        
    sm=0
    for k in data.keys():
        temp=data[k]['feedback']
        #print(temp)
        #res = json.loads(temp)
        z=0
        sm=0
        for i in temp:
            #print(i)
            z+=1
            t=TextBlob(i).sentiment.polarity
            #print(t)
            sm+=t
        if (z==0):
            rating=0
        else:
            #print(sm
            #print(z)
            rating=sm/z
        data[k]['Review_Score']=rating
        with open("products_mod_2.json", "w") as outfile: 
            json.dump(data, outfile)
        
        
        
    

In [292]:
## Done

### Price Relevance System

In [293]:
def check_price_relevence():
    with open('products_mod_2.json', 'r') as openfile: 
  
   
        data = json.load(openfile) 
    print("Specify the approx price to tune search")
    price=int(input())
    print("Specify a margin")
    margin=int(input())
    
    for k in data.keys():
        data_ref=str(data[k]['price']).replace(',','')
        temp=float(data_ref)
        
        if temp<price+margin and temp>price-margin:
            rating=1
        else:
            rating=0
            
        data[k]['Price_relevence_Score']=rating
        with open("products_mod_3.json", "w") as outfile: 
            json.dump(data, outfile)
        
        
        
    
    

### Relevence Based System

In [294]:
import pandas as pd
import ast

In [295]:
def form_featureset():
    with open('products_mod_3.json', 'r') as openfile: 
  
        
        data = json.load(openfile)
        feat=[]
        set_c=[]
        for k in data.keys():
            temp=data[k]['features']
            
            temp2=[]
        
            for i in temp:
                tag=i.split(':')[0]
                if tag not in feat:
                    feat.append(tag)
        #print(feat)
        for k in data.keys():
            temp=data[k]['features']
            temp2=[-1]*len(feat)
            for i in temp:
                tag=i.split(':')[0]
                
                #print(tag)
                
                ind= feat.index(tag)
                #print(ind)
                    
                
                temp2[ind]= i.split(':')[1]
            
                    
            set_c.append(temp2)
                
    
    df=pd.DataFrame(set_c,columns=feat)
    df.to_csv('product_descriptions.csv',index=False)
    return df
                    
                
            
            
    

In [296]:
## Done

### Merging

In [297]:


def sort_d(data):
    tot={}
    l=[]
    for k in data.keys():
         tot[k]=data[k]['Total_score']
    #print(tot)
    l.append(sorted(tot.items(), reverse=True, key = lambda x : x[1]))
    l_f=[]
    i=0
    #print((l[0])[0][1])
    while i<5:
        l_f.append(l[0][i][0])
        i+=1
    return l_f
  
        

In [298]:
def tune_search(choice):

    with open('products_mod_3.json', 'r') as openfile: 
  
        
        data = json.load(openfile)
    for k in data.keys():
            price_rel=data[k]['Price_relevence_Score']
            review_score=data[k]['Review_Score']
            pop_score=data[k]['Popularity_Score']
            pop_score_k=pop_score/4
            
            rate_score=data[k]['Rating_Score']
            rate_score_k=rate_score/5
            
            if choice==1:
                total_score=5*pop_score_k+rate_score_k+review_score+price_rel
            if choice==2:
                total_score=pop_score_k+5*rate_score_k+review_score+price_rel
            if choice==3:
                total_score=pop_score_k+rate_score_k+review_score+5*price_rel
            if choice==4:
                total_score=pop_score_k+rate_score_k+5*review_score+price_rel
                
            else:
                total_score=pop_score_k+rate_score_k+review_score+price_rel
                
            data[k]['Total_score']=total_score
            #print(data[k]['Total_score'])
    links=sort_d(data)
            
    return links
            
            

    
            
            
            
            
        

In [299]:
import json
import webbrowser
import time 
import datetime
def communicator():
    print("Specify the order")
    order=input()
    print("Any Brand You want to specify? If not say No/no")
    brand=input()
    print("Price Range? If not say No/no")
    price=input()
    
    if brand.lower()!='no':
        order_m=order+" by "+brand
    else:
        order_m=order
    if price.lower()!='no':
        order_f=order_m+" price "+price
    else:
        order_f=order_m 
    data=caller(order_f)
    
    with open("products.json", "w") as outfile: 
        json.dump(data, outfile)
    assign_popularity_rating()
    assign_sentiment_rating()
    check_price_relevence()
    df=form_featureset()
    print("Your results are ready...........")
    
    print("product_descriptions.csv has been saved.You can check the company model and features for referral later as per convinience")
    
    print("Please specify how your choices should be sorted? \n 1 for popularity based \n 2 for rating based \n 3 for price relevence based \n 4 for review based \n 5 for overall.")

    choice= int(input())
    c={1:'Popularity Based',2:'Rating Based',3:'Price Based',4:'Review Based',5:'Overall'}
    links=tune_search(choice)
    
    
    print("Here are your best 5 results")
    
    for link in links:
                
                
        webbrowser.open(link)
        
    time.sleep(5)
    options=[1,2,3,4,5]
    options.remove(choice)
    print("Here are the other bests")
    for i in options:
        links=tune_search(i)
        
        print('\n\n')
        print(c[i])
        print("\n")
        for l in links:
            print(l)
            print('\n')
        print('\n')
        
        
    
    

### Caller

In [300]:
communicator()

Specify the order
Laptop
Any Brand You want to specify? If not say No/no
Dell
Price Range? If not say No/no
Between 70000 and 80000
21
Specify the approx price to tune search
70000
Specify a margin
10000
Your results are ready...........
product_descriptions.csv has been saved.You can check the company model and features for referral later as per convinience
Please specify how your choices should be sorted? 
 1 for popularity based 
 2 for rating based 
 3 for price relevence based 
 4 for review based 
 5 for overall.
5
Here are your best 5 results
Here are the other bests



Popularity Based


https://www.amazon.in/Dell-Inspiron-5370-13-3-inch-Graphics/dp/B07B2W7DCB/ref=sr_1_4?dchild=1&keywords=Laptop+by+Dell+price+Between+70000+and+80000&qid=1595324585&sr=8-4


https://www.amazon.in/Inspiron-5370-13-3-inch-i7-8550U-Graphics/dp/B07B6K4YM6/ref=sr_1_12?dchild=1&keywords=Laptop+by+Dell+price+Between+70000+and+80000&qid=1595324585&sr=8-12


https://www.amazon.in/Inspiron-5593-15-6-inch-i